In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.metrics import mean_absolute_error
import os
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import Ridge, Lasso, ElasticNet

import csv
import os
import json
import numpy as np
from datetime import datetime

In [11]:

folder = '../data/processed'
filename = 'dataset_final.csv'
file_path = os.path.join(folder, filename)

try:
    df = pd.read_csv(
        file_path,
        sep=',',
        decimal='.'
    )
    df = df.sort_values(by='DIA', ignore_index=True)
    print(f"✅ DataFrame cargado exitosamente desde: {file_path}")
    print(f"Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    print(df.head())

except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo en la ruta: {file_path}")
except Exception as e:
    print(f"❌ Error al leer el archivo: {e}")

✅ DataFrame cargado exitosamente desde: ../data/processed\dataset_final.csv
Dimensiones: (1184, 304)

Primeras 5 filas:
          DIA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  \
0  2020-07-01      642.727209            47.145349       69.023256   
1  2020-07-02        7.767254             0.769609        0.798838   
2  2020-07-03        8.801205             0.862593        0.835762   
3  2020-07-04        5.175639             0.439225        0.371077   
4  2020-07-05        7.924665             0.802365        0.717787   

   EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  \
0        0.000000         13.813953        14.578784         0.000000   
1        0.319229          2.358593         4.158962         1.506838   
2        0.260924          1.985462        39.076667         1.448962   
3        0.258048          1.442114         4.348182         1.355238   
4        0.301592          1.664726         5.125920         2.704348   

   EE Linea 4 / Hl  

In [12]:

# 1️⃣ Cargar dataset definitivo
ruta_csv = "../data/processed/X_test_preproc.csv"
X_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/X_train_preproc.csv"
X_train = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_test.csv"
y_test = pd.read_csv(ruta_csv, sep=',', decimal='.')
ruta_csv = "../data/processed/y_train.csv"
y_train = pd.read_csv(ruta_csv, sep=',', decimal='.')


print("Shape dataset:", df.shape)




Shape dataset: (1184, 304)


In [13]:
def objective_rf_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_float("max_features", 0.1, 1.0, log=True)

    # Modelo
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        n_jobs=-1,
        random_state=42
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [14]:
def objective_xgb_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbosity=0
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [15]:
def objective_lgbm_mae(trial):
    # Hiperparámetros
    n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    num_leaves = trial.suggest_int("num_leaves", 20, 150)
    subsample = trial.suggest_float("subsample", 0.6, 1.0, step=0.1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True) # L1
    reg_lambda = trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True) # L2

    # Modelo
    model = lgb.LGBMRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        num_leaves=num_leaves,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        n_jobs=-1,
        random_state=42,
        verbose=-1
    )
    
    # CV y Métrica (MAE)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

In [16]:
def objective_elasticnet_mae(trial):
    alpha = trial.suggest_float("alpha", 1e-3, 10.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0) 

    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42, max_iter=2000)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="neg_mean_absolute_error", n_jobs=-1)
    
    return -np.mean(scores)

# 1. Crear los estudios (todos buscan minimizar el MAE)
study_rf_mae = optuna.create_study(direction="minimize")
study_xgb_mae = optuna.create_study(direction="minimize")
study_lgbm_mae = optuna.create_study(direction="minimize")
study_elasticnet_mae = optuna.create_study(direction="minimize")

# 2. Ejecutar las optimizaciones
print("Optimizando Random Forest (MAE)...")
study_rf_mae.optimize(objective_rf_mae, n_trials=50)

print("Optimizando XGBoost (MAE)...")
study_xgb_mae.optimize(objective_xgb_mae, n_trials=75)

print("Optimizando LightGBM (MAE)...")
study_lgbm_mae.optimize(objective_lgbm_mae, n_trials=75)

print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=40)

# 3. Ver los mejores parámetros
print(f"Mejor RF (MAE): {study_rf_mae.best_value} con params {study_rf_mae.best_params}")
print(f"Mejor XGB (MAE): {study_xgb_mae.best_value} con params {study_xgb_mae.best_params}")
print(f"Mejor LGBM (MAE): {study_lgbm_mae.best_value} con params {study_lgbm_mae.best_params}")
print(f"Mejor ElasticNet (MAE): {study_elasticnet_mae.best_value} con params {study_elasticnet_mae.best_params}")

In [17]:

def log_experiment(model_name: str, 
                   model_object: object, 
                   study: optuna.study.Study, 
                   X_train, y_train, X_test, y_test):
    """
    Entrena un modelo con los mejores parámetros de un estudio de Optuna,
    calcula métricas y las guarda en un CSV.
    """
    
    # --- 1. Definir la ruta del log ---
    log_dir = "results"
    log_file = os.path.join(log_dir, "experiment_logs.csv")
    os.makedirs(log_dir, exist_ok=True) # Crea la carpeta si no existe

    # --- 2. Obtener mejores parámetros y valor del estudio ---
    best_params = study.best_params
    best_value = study.best_value # El MAE de CV que encontró Optuna
    
    # --- 3. Instanciar y entrenar el modelo final ---
    # Manejo especial para modelos lineales que necesitan Pipeline
    if "pipeline" in model_name.lower():
        # El pipeline ya está definido en la función objective, 
        # aquí solo re-creamos el mejor.
        if model_name == "Pipeline_ElasticNet":
             # Re-crea el pipeline con los mejores params
             model =ElasticNet(
                    alpha=best_params.get('alpha'), 
                    l1_ratio=best_params.get('l1_ratio'),
                    random_state=42, 
                    max_iter=2000
                )
            
            
    else:
        # Para modelos no-pipeline (RF, XGB, LGBM)
        model = model_object(**best_params, random_state=42, n_jobs=-1)
        # Ajustes para modelos que no les gusta 'n_jobs' o necesitan 'verbose'
        if isinstance(model, (xgb.XGBRegressor, lgb.LGBMRegressor)):
            model.set_params(verbosity=0) if isinstance(model, xgb.XGBRegressor) else None
            model.set_params(verbose=-1) if isinstance(model, lgb.LGBMRegressor) else None

    # Entrenar el modelo final con todos los datos de train
    model.fit(X_train, y_train)

    # --- 4. Calcular Métricas (Train y Test) ---
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    metrics = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_name': model_name,
        'optuna_cv_mae': best_value, # Métrica optimizada
        'mae_train': mean_absolute_error(y_train, y_pred_train),
        'mae_test': mean_absolute_error(y_test, y_pred_test),
        'rmse_train': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'rmse_test': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'r2_train': r2_score(y_train, y_pred_train),
        'r2_test': r2_score(y_test, y_pred_test),
        'hyperparameters': json.dumps(best_params) # Guardar params como string JSON
    }
    
    # --- 5. Escribir en el CSV ---
    file_exists = os.path.isfile(log_file)
    
    with open(log_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=metrics.keys())
        
        if not file_exists:
            writer.writeheader() # Escribir cabecera solo si el archivo es nuevo
            
        writer.writerow(metrics)
        
    print(f"Resultados de '{model_name}' guardados en {log_file}")

In [ ]:
# --- 1. Ejecutar el estudio (ejemplo con Random Forest) ---
print("Optimizando Random Forest (MAE)...")
study_rf_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_rf_mae' está definida)
study_rf_mae.optimize(objective_rf_mae, n_trials=15) 

# --- 2. Registrar el resultado ---
log_experiment(
    model_name="RandomForest",
    model_object=RandomForestRegressor, # Pasa la clase del modelo
    study=study_rf_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 5. Ejecutar el estudio (ejemplo con ElasticNet) ---
print("Optimizando ElasticNet (MAE)...")
study_elasticnet_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_elasticnet_mae' está definida)
study_elasticnet_mae.optimize(objective_elasticnet_mae, n_trials=30)

# --- 6. Registrar el resultado ---
# Nota: 'model_object' es None porque se maneja dentro del 'if'
log_experiment(
    model_name="Pipeline_ElasticNet",
    model_object=None, 
    study=study_elasticnet_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# --- 3. Ejecutar el estudio (ejemplo con XGBoost) ---
print("Optimizando XGBoost (MAE)...")
study_xgb_mae = optuna.create_study(direction="minimize")
# (Aquí asumimos que 'objective_xgb_mae' está definida)
study_xgb_mae.optimize(objective_xgb_mae, n_trials=5)

# --- 4. Registrar el resultado ---
log_experiment(
    model_name="XGBoost",
    model_object=xgb.XGBRegressor, # Pasa la clase del modelo
    study=study_xgb_mae,
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test
)

# Repetir para LightGBM...

[I 2025-11-15 14:08:16,022] A new study created in memory with name: no-name-d8384e7c-437c-4df9-8ca0-3d8a4cb36f44


Optimizando Random Forest (MAE)...


[I 2025-11-15 14:08:26,220] Trial 0 finished with value: 9132.613083411647 and parameters: {'n_estimators': 450, 'max_depth': 31, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 0.24636527742907252}. Best is trial 0 with value: 9132.613083411647.
[I 2025-11-15 14:08:31,246] Trial 1 finished with value: 10588.828433003382 and parameters: {'n_estimators': 150, 'max_depth': 13, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 0.16449629918650788}. Best is trial 0 with value: 9132.613083411647.
[I 2025-11-15 14:08:33,713] Trial 2 finished with value: 9468.753411903132 and parameters: {'n_estimators': 250, 'max_depth': 9, 'min_samples_split': 12, 'min_samples_leaf': 7, 'max_features': 0.26613661575955216}. Best is trial 0 with value: 9132.613083411647.
[I 2025-11-15 14:08:45,080] Trial 3 finished with value: 9567.22105795698 and parameters: {'n_estimators': 800, 'max_depth': 38, 'min_samples_split': 16, 'min_samples_leaf': 10, 'max_features': 0.486360333479090

Resultados de 'RandomForest' guardados en results\experiment_logs.csv
Optimizando ElasticNet (MAE)...


[I 2025-11-15 14:09:08,360] Trial 0 finished with value: 15111.201374012448 and parameters: {'alpha': 0.2634886355954175, 'l1_ratio': 0.3029687571186219}. Best is trial 0 with value: 15111.201374012448.
[I 2025-11-15 14:09:08,621] Trial 1 finished with value: 14657.6732133451 and parameters: {'alpha': 0.22297663436474036, 'l1_ratio': 0.06139370902019958}. Best is trial 1 with value: 14657.6732133451.
[I 2025-11-15 14:09:08,841] Trial 2 finished with value: 28490.652469442204 and parameters: {'alpha': 0.001934597825688664, 'l1_ratio': 0.7755079057073868}. Best is trial 1 with value: 14657.6732133451.
[I 2025-11-15 14:09:09,079] Trial 3 finished with value: 23187.69058010725 and parameters: {'alpha': 0.02464041082788516, 'l1_ratio': 0.5214786898245859}. Best is trial 1 with value: 14657.6732133451.
[I 2025-11-15 14:09:09,371] Trial 4 finished with value: 23911.072788696325 and parameters: {'alpha': 0.01605227391965591, 'l1_ratio': 0.5497746836917935}. Best is trial 1 with value: 14657.67

Resultados de 'Pipeline_ElasticNet' guardados en results\experiment_logs.csv
Optimizando XGBoost (MAE)...


[I 2025-11-15 14:09:41,896] Trial 0 finished with value: 12220.52314453125 and parameters: {'n_estimators': 900, 'learning_rate': 0.1904493667018738, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 3.898470237330194e-06, 'reg_lambda': 0.13988062844855642}. Best is trial 0 with value: 12220.52314453125.
[I 2025-11-15 14:11:21,957] Trial 1 finished with value: 11612.04189453125 and parameters: {'n_estimators': 900, 'learning_rate': 0.04906289733080212, 'max_depth': 10, 'subsample': 0.9, 'colsample_bytree': 1.0, 'reg_alpha': 0.00014699320990922487, 'reg_lambda': 5.1661014543360604e-08}. Best is trial 1 with value: 11612.04189453125.
[I 2025-11-15 14:11:27,838] Trial 2 finished with value: 11607.59765625 and parameters: {'n_estimators': 600, 'learning_rate': 0.015195406702123685, 'max_depth': 4, 'subsample': 0.6, 'colsample_bytree': 0.8, 'reg_alpha': 0.001463912294169605, 'reg_lambda': 0.000573071664279915}. Best is trial 2 with value: 11607.59765625.


Resultados de 'XGBoost' guardados en results\experiment_logs.csv
